In [ ]:
# Uncomment the command below if you run this script separately. 

#%run AnalysisFunctions.ipynb


In [ ]:
#filtered = ""

In [ ]:
print("Running Ito Analysis")


print(f"Apply Filter?: {apply_filter}")

if apply_filter.lower() == "true":
    filtered = "/Filtered"
    print("Compiling results with filtered pairs")
else:
    filtered = ""
    print("Compiling results with unfiltered pairs")


In [ ]:
## This screen has some inconsistences with the gene names, so convert them to Entrez ID before analysis.

def Convert_Symbols_to_Entrez(entrez_dict, df):
    df = reindex_alphbetically(df)
    df = df.rename_axis('GENE')
    df = df.reset_index()
    df[['gene1', 'gene2']] = df['GENE'].str.split('_', expand=True)
    df['gene1_Entrez_ID'] = df['gene1'].map(entrez_dict)
    df['gene2_Entrez_ID'] = df['gene2'].map(entrez_dict)

    missing = df[df['gene1_Entrez_ID'].isna() | df['gene2_Entrez_ID'].isna()].shape
    print("missing:")
    print(missing)
    
    df['gene_pair']  = df['gene1_Entrez_ID']+ "_" + df['gene2_Entrez_ID']
    df = df.set_index('gene_pair')
    df = df.drop(columns = ['gene1','gene2','gene1_Entrez_ID','gene2_Entrez_ID', 'GENE'])
    
    df = df[df.index.notna()]
    return(df)

In [ ]:
# Gene name to Entrez ID data dowloaded from https://www.genenames.org/
dtype_specification = {'NCBI Gene ID': str}
genenames = pd.read_csv('../InputData/genenames.txt', delimiter = "\t",
                        dtype=dtype_specification)
genenames

genenames = genenames[['Approved symbol', 'NCBI Gene ID']]
genenames = genenames.rename(columns = {'NCBI Gene ID':'Entrez ID'})

genenames = genenames.dropna(subset=['Entrez ID'])


entrez_dict = dict(zip(genenames['Approved symbol'], genenames['Entrez ID']))


In [ ]:
ground_truth_depmap_hit_entrez = Convert_Symbols_to_Entrez(entrez_dict, ground_truth_depmap_hit)
Koferle_entrez = Convert_Symbols_to_Entrez(entrez_dict, Koferle)

In [ ]:
# zdLFC doesnt drop ny pairs so filtered or unfiltered doesnt make a difference 
zdLFC = pd.read_csv("../zdLFC Scripts/zdLFC Output/ITO.csv")
zdLFC = pd.read_csv("C:\\Users\\Hamda\\OneDrive\\Documents\\GitHub\\PostDoc\\Conway\\Research\\GIScoring\\Hamda's Work\\zdLFC\\zdLFC_Output\\ITO.csv")
zdLFC = zdLFC.rename(columns={zdLFC.columns[0]: 'gene_pair'})
zdLFC = zdLFC.set_index('gene_pair')
#zdLFC
zdLFC = Convert_Symbols_to_Entrez(entrez_dict, zdLFC)
zdLFC = zdLFC * -1
zdLFC ## missing are due to entrez mapping, not due to score

In [ ]:
orthrus = pd.read_csv("../Orthrus Scripts/OrthrusOutput" + filtered + "/ito_orthrus.tsv", sep='\t')
#display(orthrus)
orthrus = orthrus.filter(like='differential', axis=1) \
                    .join(orthrus.filter(items=['gene1','gene2']))
orthrus.columns = orthrus.columns.str.replace('differential_combn_vs_single_', '')

orthrus['gene_pair'] = orthrus['gene1'] + "_" +orthrus['gene2']
orthrus = orthrus.set_index('gene_pair')
orthrus.drop(columns = ['gene1', 'gene2'], inplace = True)
orthrus = Convert_Symbols_to_Entrez(entrez_dict, orthrus)
#count_nan_rows = a.index.isna().sum()
#count_nan_rows # 104 rows are missing
orthrus = orthrus * -1
orthrus

In [ ]:
gemini_SensL = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_ITO_Sensitive.csv")
gemini_SensL.rename({'Unnamed: 0':'gene_pair'}, inplace = True, axis = 1)
gemini_SensL['gene_pair'] =gemini_SensL['gene_pair'].str.replace(';', '_')
gemini_SensL.set_index('gene_pair', inplace = True)
#display(gemini_SensL[gemini_SensL.isnull().any(axis=1)])

gemini_SensL = Convert_Symbols_to_Entrez(entrez_dict, gemini_SensL)




In [ ]:
gemini_Strong = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_ITO_Strong.csv")
gemini_Strong.rename({'Unnamed: 0':'gene_pair'}, inplace = True, axis = 1)
gemini_Strong['gene_pair'] =gemini_Strong['gene_pair'].str.replace(';', '_')
gemini_Strong.set_index('gene_pair', inplace = True)
gemini_Strong = Convert_Symbols_to_Entrez(entrez_dict, gemini_Strong)

gemini_Strong

In [ ]:
# Parrish doesnt drop any pairs either 
parrish_score = pd.read_csv('../Parrish Score Scripts/ParrishOutput/ito_parrish.csv')
parrish_score.set_index('paralog_pair', inplace=True)
parrish_score = Convert_Symbols_to_Entrez(entrez_dict, parrish_score)
parrish_score = parrish_score * -1
parrish_score

In [ ]:
Ito_A549 = pd.concat([zdLFC['A549'],
                      orthrus['A549'],
                      gemini_Strong['A549'],
                      gemini_SensL['A549'],
                      parrish_score['GI_score_A549']
                      ], axis = 1, keys=scores)



results_ito_A549 = Cell_Line_Analysis(Ito_A549, 'A549', 'Ito', 
                   scores, ground_truth_depmap_hit_entrez)


In [ ]:
Ito_Meljuso = pd.concat([zdLFC['Meljuso'],
                      orthrus['Meljuso'],
                      gemini_Strong['Meljuso'],
                      gemini_SensL['Meljuso'],
                      parrish_score['GI_score_Meljuso']
                      ], keys= scores, axis = 1)


results_ito_Meljuso = Cell_Line_Analysis(Ito_Meljuso, 'Meljuso', 'Ito', 
                   scores, ground_truth_depmap_hit_entrez)


In [ ]:
Ito_GI1 = pd.concat([zdLFC['GI1_004'],
                      orthrus['GT1'],
                      gemini_Strong['GI1_004'],
                      gemini_SensL['GI1_004'],
                      parrish_score['GI_score_GI1']

                      ], keys= scores, axis = 1)


results_ito_GI1 = Cell_Line_Analysis(Ito_GI1, 'GI1', 'Ito', 
                   scores, ground_truth_depmap_hit_entrez)


In [ ]:
Ito_HS936T = pd.concat([zdLFC['HS936T'],
                      orthrus['HS936T'],
                      gemini_Strong['HS936T'],
                      gemini_SensL['HS936T'],
                      parrish_score['GI_score_HS936T']
                      ], keys= scores, axis = 1)


results_ito_HS936T = Cell_Line_Analysis(Ito_HS936T, 'HS936T', 'Ito', 
                   scores, ground_truth_depmap_hit_entrez)


In [ ]:
Ito_HS944T = pd.concat([zdLFC['HS944T'],
                      orthrus['HS944T'],
                      gemini_Strong['HS944T'],
                      gemini_SensL['HS944T'],
                      parrish_score['GI_score_HS944T']
                      ], keys= scores, axis = 1)


results_ito_HS944T = Cell_Line_Analysis(Ito_HS944T, 'HS944T', 'Ito', 
                   scores, ground_truth_depmap_hit_entrez)


In [ ]:
Ito_HSC5 = pd.concat([zdLFC['HSC5'],
                      orthrus['HSC5'],
                      gemini_Strong['HSC5'],
                      gemini_SensL['HSC5'],
                      parrish_score['GI_score_HSC5']
                      ], keys= scores, axis = 1)


results_ito_HSC5 = Cell_Line_Analysis(Ito_HSC5, 'HSC5', 'Ito', 
                   scores, ground_truth_depmap_hit_entrez)


In [ ]:
Ito_IPC298 = pd.concat([zdLFC['IPC298'],
                      orthrus['IPC298'],
                      gemini_Strong['IPC298'],
                      gemini_SensL['IPC298'],
                      parrish_score['GI_score_IPC298']
                      ], keys= scores, axis = 1)


results_ito_IPC298 = Cell_Line_Analysis(Ito_IPC298, 'IPC298', 'Ito', 
                   scores, ground_truth_depmap_hit_entrez)


In [ ]:
Ito_MEL202_003 = pd.concat([zdLFC['MEL202_003'],
                      orthrus['MEL202_203'],
                      gemini_Strong['MEL202_003'],
                      gemini_SensL['MEL202_003'],
                      parrish_score['GI_score_MEL202']
                      ], keys= scores, axis = 1)


results_ito_MEL202_003 = Cell_Line_Analysis(Ito_MEL202_003, 'MEL202_003', 'Ito', 
                   scores, ground_truth_depmap_hit_entrez)


In [ ]:
Ito_MEWO = pd.concat([zdLFC['MEWO'],
                      orthrus['MEWO'],
                      gemini_Strong['MEWO'],
                      gemini_SensL['MEWO'],
                      parrish_score['GI_score_MEWO']
                      ], keys= scores, axis = 1)


results_ito_MEWO = Cell_Line_Analysis(Ito_MEWO, 'MEWO', 'Ito', 
                   scores, ground_truth_depmap_hit_entrez)




In [ ]:
Ito_PATU8988S = pd.concat([zdLFC['PATU8988S'],
                      orthrus['PATU8988S'],
                      gemini_Strong['PATU8988S'],
                      gemini_SensL['PATU8988S'],
                      parrish_score['GI_score_PATU8988S']
                      ], keys= scores, axis = 1)


results_ito_PATU8988S = Cell_Line_Analysis(Ito_PATU8988S, 'PATU8988S', 'Ito', 
                   scores, ground_truth_depmap_hit_entrez)


In [ ]:
Ito_PK1 = pd.concat([zdLFC['PK1'],
                      orthrus['PK1'],
                      gemini_Strong['PK1'],
                      gemini_SensL['PK1'],
                      parrish_score['GI_score_PK1']
                      ], keys= scores, axis = 1)


results_ito_PK1 = Cell_Line_Analysis(Ito_PK1, 'PK1', 'Ito', 
                   scores, ground_truth_depmap_hit_entrez)


In [ ]:
full_list = [results_ito_A549, results_ito_GI1, results_ito_HS936T,
             results_ito_HS944T, results_ito_HSC5, results_ito_IPC298,
             results_ito_MEL202_003, results_ito_Meljuso, results_ito_PATU8988S, results_ito_PK1, results_ito_MEWO]

roc, pr = ConvertResultsToDF(full_list )

## Write results to a file
roc.to_csv("Output" + filtered + "/Ito_ROC_DepMapHits.csv", index=False)
pr.to_csv("Output"  + filtered + "/Ito_PR_DepMapHits.csv", index=False)


In [ ]:
ito_all = pd.concat([Ito_A549, Ito_GI1, Ito_HS936T, 
                     Ito_HS944T, Ito_HSC5, Ito_IPC298, 
                     Ito_MEL202_003, Ito_Meljuso, Ito_MEWO, 
                     Ito_PATU8988S, Ito_PK1])
ito_all
results_ito_all = Cell_Line_Analysis(ito_all, 'All', 'Ito', scores, ground_truth_depmap_hit_entrez)
roc_all, pr_all = ConvertResultsToDF([results_ito_all] )
## Write results to a file
roc_all.to_csv("Output" + filtered + "/Ito_All_ROC_DepMapHits.csv", index=False)
pr_all.to_csv("Output" + filtered + "/Ito_All_PR_DepMapHits.csv", index=False)


In [ ]:

results_ito_all_Koferle = Cell_Line_Analysis(ito_all, 'All', 'Ito', scores, Koferle_entrez)
roc_all_Koferle, pr_all_Koferle  = ConvertResultsToDF([results_ito_all_Koferle] )
## Write results to a file
roc_all_Koferle.to_csv("Output" + filtered + "/Ito_All_ROC_Koferle.csv", index=False)
pr_all_Koferle.to_csv("Output" + filtered+ "/Ito_All_PR_Koferle.csv", index=False)


In [ ]:
results_ito_A549_Koferle = Cell_Line_Analysis(Ito_A549, 'A549', 'Ito', 
                   scores, Koferle_entrez)
results_ito_Meljuso_Koferle = Cell_Line_Analysis(Ito_Meljuso, 'Meljuso', 'Ito', 
                   scores, Koferle_entrez)
results_ito_GI1_Koferle = Cell_Line_Analysis(Ito_GI1, 'GI1', 'Ito', 
                   scores, Koferle_entrez)
results_ito_HS936T_Koferle = Cell_Line_Analysis(Ito_HS936T, 'HS936T', 'Ito', 
                   scores, Koferle_entrez)
results_ito_HS944T_Koferle = Cell_Line_Analysis(Ito_HS944T, 'HS944T', 'Ito', 
                   scores, Koferle_entrez)
results_ito_HSC5_Koferle = Cell_Line_Analysis(Ito_HSC5, 'HSC5', 'Ito', 
                   scores, Koferle_entrez)
results_ito_IPC298_Koferle = Cell_Line_Analysis(Ito_IPC298, 'IPC298', 'Ito', 
                   scores, Koferle_entrez)
results_ito_MEL202_003_Koferle = Cell_Line_Analysis(Ito_MEL202_003, 'MEL202_003', 'Ito', 
                   scores, Koferle_entrez)
results_ito_MEWO_Koferle = Cell_Line_Analysis(Ito_MEWO, 'MEWO', 'Ito', 
                   scores, Koferle_entrez)
results_ito_PATU8988S_Koferle = Cell_Line_Analysis(Ito_PATU8988S, 'PATU8988S', 'Ito', 
                   scores, Koferle_entrez)
results_ito_PK1_Koferle = Cell_Line_Analysis(Ito_PK1, 'PK1', 'Ito', 
                   scores, Koferle_entrez)


full_list = [results_ito_A549_Koferle, results_ito_Meljuso_Koferle, results_ito_GI1_Koferle,
             results_ito_HS936T_Koferle, results_ito_HS944T_Koferle, results_ito_HSC5_Koferle,
             results_ito_IPC298_Koferle, results_ito_MEL202_003_Koferle, results_ito_MEWO_Koferle, 
             results_ito_PATU8988S_Koferle, results_ito_PK1_Koferle]

roc_Koferle, pr_Koferle = ConvertResultsToDF(full_list )

## Write results to a file
roc_Koferle.to_csv("Output" + filtered + "/Ito_ROC_Koferle.csv", index=False)
pr_Koferle.to_csv("Output" + filtered + "/Ito_PR_Koferle.csv", index=False)


In [ ]:

output_path = "Output" + filtered + "\Comparison"
ito_all.corr(method = "spearman").to_csv(output_path + "\Ito_Correlations.csv")
Ito_A549_J = jaccard_similarity(Ito_A549, 0.95)
Ito_GI1_J = jaccard_similarity(Ito_GI1, 0.95)
Ito_HS936T_J = jaccard_similarity(Ito_HS936T, 0.95)
Ito_HS944T_J = jaccard_similarity(Ito_HS944T, 0.95)
Ito_HSC5_J = jaccard_similarity(Ito_HSC5, 0.95)
Ito_IPC298_J = jaccard_similarity(Ito_IPC298, 0.95)
Ito_MEL202_003_J = jaccard_similarity(Ito_MEL202_003, 0.95)
Ito_Meljuso_J = jaccard_similarity(Ito_Meljuso, 0.95)
Ito_MEWO_J = jaccard_similarity(Ito_MEWO, 0.95)
Ito_PATU8988S_J = jaccard_similarity(Ito_PATU8988S, 0.95)
Ito_PK1_J = jaccard_similarity(Ito_PK1, 0.95)



In [ ]:
Ito_A549_J.to_csv(output_path + "\Ito_A549_Jaccard.csv")
Ito_GI1_J.to_csv(output_path + "\Ito_GI1_Jaccard.csv")
Ito_HS936T_J.to_csv(output_path + "\Ito_HS936T_Jaccard.csv")
Ito_HS944T_J.to_csv(output_path + "\Ito_HS944T_Jaccard.csv")
Ito_HSC5_J.to_csv(output_path + "\Ito_HSC5_Jaccard.csv")
Ito_IPC298_J.to_csv(output_path + "\Ito_IPC298_Jaccard.csv")
Ito_MEL202_003_J.to_csv(output_path + "\Ito_MEL202_003_Jaccard.csv")

Ito_Meljuso_J.to_csv(output_path + "\Ito_Meljuso_Jaccard.csv")# = jaccard_similarity(Ito_Meljuso, 0.95)
Ito_MEWO_J.to_csv(output_path + "\Ito_MEWO_Jaccard.csv") #= jaccard_similarity(Ito_MEWO, 0.95)
Ito_PATU8988S_J.to_csv(output_path + "\Ito_PATU8988S_Jaccard.csv") # = jaccard_similarity(Ito_PATU8988S, 0.95)
Ito_PK1_J.to_csv(output_path + "\Ito_PK2_Jaccard.csv") #= jaccard_similarity(Ito_PK1, 0.95)

Ito_Jaccard = (Ito_A549_J + Ito_GI1_J + Ito_HS936T_J + Ito_HS944T_J + Ito_HSC5_J + Ito_IPC298_J + 
    Ito_MEL202_003_J + Ito_Meljuso_J + Ito_MEWO_J + Ito_PATU8988S_J + Ito_PK1_J )/11
Ito_Jaccard.to_csv(output_path + "\Ito_Jaccard.csv")